In [1]:
import pandas as pd
import numpy as np
import pickle
from glob import glob
from collections import defaultdict
import os

In [2]:
def chunks(lst, n):
    chunked = [ lst[i:i + n] for i in range(0, len(lst), n) ]
    return np.array(chunked[:-1])

In [5]:
n = 32
normalize = True
dirs = [x[0] for x in os.walk("../../raw/ADL_HMP_Dataset/") if "MODEL" not in x[0]][1:]

data = defaultdict(lambda: defaultdict(list))
dico = {}

for c, dire in enumerate(dirs):
    files = glob(f"{dire}/*")
    files = sorted(files)
    subjects = [f.split("-")[-1].split(".")[0] for f in files]

    for s, fname in zip(subjects, files):

        values = pd.read_csv(fname, header=None, sep=" ").dropna().values.astype(float)

        if normalize:
            #values = (values - values.mean(axis=0)) / values.std(axis=0)
            values = values / ( values.max(axis=0) - values.min(axis=0) )
        
        X_ = [ arr.T for arr in chunks(values, n) ]

        data[s][c] += X_

        #print(s, fname, np.array(X_).shape, np.array(data[s][c]).shape)
data = { i: {c: np.array(X) for c, X in data[s].items()} for i, s in enumerate(data.keys()) if len(data[s]) > 10 }

In [6]:
for ke in data:
    #print(data[ke].keys())
    print("==============>", ke, len(data[ke].keys()))

==============> 0 14
==============> 1 12
==============> 2 12


In [7]:
with open(f"data.pickle", 'wb') as handle: 
    pickle.dump(data, handle)

In [6]:
with open(f"data.pickle", 'rb') as handle:
    data = pickle.load(handle)
    print( data.keys() )
    print( data[1].keys() )

dict_keys([0, 1, 2])
dict_keys([0, 1, 2, 3, 4, 7, 8, 9, 10, 11, 12, 13])
